In [3]:
import csv
import pandas as pd
import sqlite3
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

#### Read the data from csv's if it has already been converted so such.  Else, ignore this line and start with converting osm -> csv

In [2]:
nodes = pd.read_csv("nodes.csv")
nodes_tags = pd.read_csv("nodes_tags.csv")
ways = pd.read_csv('ways.csv')
ways_nodes = pd.read_csv("ways_nodes.csv")
ways_tags = pd.read_csv("ways_tags.csv")

# Download OpenStreemMap XML file to local file using web api overpass-api.de query form.

In [12]:
"""from data.py
This will extract fields from the OpenStreetMap XML file and store various field values in ways and nodes CSV files.
Parsing of field values includig ':' and separately problem characters (not a-z,:) is done.  The field vlaues that are 
extracted are included in the 'NODE_FIELDS','NODE_TAGS_PATH','WAYS_PATH','WAY_NODES_PATH','WAY_TAGS_PATH' lists.  The 
schema is validated as well.

A small subset of the data was validated with the provided function and then validation set to False for larger data set.
"""

OSM_PATH = "Phx_metro_smaller.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

#create a regex that is any lower case letters : any lower cases letters
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
#create a regex of the troublesome characters that will make a tag be ignored in parsing
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    
    if element.tag == 'node':
        return parse_node(element)
            
    elif element.tag == 'way':
        return parse_way(element)
  
    
def parse_node(element):
    #Parse an element.tag = node component
    node_attribs = {}
    tags = []
    for att in NODE_FIELDS:
        node_attribs[att] = element.attrib[att]
        
    #build node_tags
    position = 0
    for child in element:
        if child.tag == 'tag':
            tag = parse_tag(element, child)
            tags.append(tag)
    return {'node': node_attribs, 'node_tags' : tags}    


def parse_way(element):
    #Parse an element.tag = way component
    way_attribs = {}
    way_nodes = []
    tags = []
    #build top level way attributes
    for att in WAY_FIELDS:
        way_attribs[att] = element.attrib[att]
    
    #build way_tags
    position = 0
    for child in element:
        if child.tag == 'tag':
            tag = parse_tag(element, child)
            tags.append(tag)
        
        if child.tag == 'nd':
            #parse nd tag
            nd, position = parse_nd(element, child, position)
            way_nodes.append(nd)
        
    return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


def parse_nd(element, child, position):
    #Parse component when child.tag = nd
    nd = {}
    nd['id'] = element.attrib['id']
    nd['node_id'] = child.attrib['ref']
    nd['position'] = position
    position += 1
    return nd, position

def parse_tag(element, child):
    #This parses both the way and node ``tag'' tag
    tag = {}
    tag['id'] = element.attrib['id']
    
    #parse k attrib with a colon or two
    if bool(LOWER_COLON.search(child.attrib['k'])):
        k = child.attrib['k'].split(':')
        if len(k) == 3:
            tag['key']=k[1]+':'+k[2]
            
        elif len(k) == 2:
            tag['key'] = k[1]
        tag['type']= k[0]
        tag['value'] = child.attrib['v'] 
                
    elif not bool(PROBLEMCHARS.search(child.attrib['k'])):
        tag['type'] = 'regular'
        tag['value']=child.attrib['v']
        tag['key'] = child.attrib['k']
            
    return tag





# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)

In [16]:
def db_table_from_csv(db_name, file_name, table_name, keys = [], key_params = []):
    '''file_name: name of csv file, 
    table: name of table to create in db_name, 
    keys: columsn of .csv file_name to include in db table
    key_params: parameters for data type for each field in table'''

    con = sqlite3.connect(db_name)
    cur = con.cursor()

    #Creation of nodes ways table
    cur.execute("DROP TABLE IF EXISTS " + table_name + ";")
    con.commit()
    
    create_str = "" #holds schema string to pass to sql
    for i in range(len(keys)):
        create_str = create_str + keys[i] + " " + key_params[i]+","
    create_str = create_str[:-1]
    cur.execute("CREATE TABLE " + table_name + "("+create_str+");")

    con.commit()
    with open(file_name, 'rb') as fin:
        dr = csv.DictReader(fin)
        to_db = [[i[key].decode("utf-8") for key in keys] for i in dr]
    
    key_str = ""
    key_qs = ""
    for key in keys:
        key_str = key_str+key+","
        key_qs = key_qs + "?," #just how many fields to enter into db
    key_qs = key_qs[:-1] #strip comma at end
    key_str=key_str[:-1]
    
    #insert data into db according to keys
    cur.executemany("INSERT INTO "+table_name+"("+key_str+") VALUES ("+key_qs+");", to_db)
    con.commit()

    con.close()
    return

In [17]:
DB_NAME = 'Phx_metro_smaller.db'
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

NODE_PARAMS = ['INTEGER PRIMARY KEY NOT NULL', 'REAL', 'REAL', 'TEXT', 'INTEGER', 'TEXT', 'INTEGER', 'DATE']
NODE_TAGS_PARAMS = ['INTEGER NOT NULL', 'TEXT NOT NULL', 'TEXT NOT NULL', 'TEXT NOT NULL']
WAY_PARAMS = ['INTEGER NOT NULL', 'TEXT NOT NULL', 'INTEGER NOT NULL', 'TEXT NOT NULL', 'INTEGER NOT NULL', 'TEXT NOT NULL']
WAY_NODES_PARAMS = ['INTEGER NOT NULL', 'INTEGER NOT NULL', 'INTEGER NOT NULL']
WAY_TAGS_PARAMS = ['INTEGER NOT NULL', 'TEXT NOT NULL', 'TEXT NOT NULL', 'TEXT NOT NULL']


db_table_from_csv(DB_NAME, 'nodes.csv', 'nodes', NODE_FIELDS, NODE_PARAMS)
db_table_from_csv(DB_NAME, 'nodes_tags.csv', 'nodes_tags', NODE_TAGS_FIELDS, NODE_TAGS_PARAMS)
db_table_from_csv(DB_NAME, 'ways.csv', 'ways', WAY_FIELDS, WAY_PARAMS)
db_table_from_csv(DB_NAME, 'ways_tags.csv', 'way_tags', WAY_TAGS_FIELDS, WAY_TAGS_PARAMS)
db_table_from_csv(DB_NAME, 'ways_nodes.csv', 'way_nodes', WAY_NODES_FIELDS, WAY_NODES_PARAMS)

#### Cities list from http://phoenix.about.com/od/govtcity/qt/cities-towns-maricopa-county.htm

In [10]:
cities = ['Apache Junction','Avondale','Buckeye','Carefree','Cave Creek','Chandler','El Mirage','Fountain Hills',
          'Gila Bend','Gilbert','Glendale','Goodyear','Guadalupe','Litchfield Park','Mesa','Paradise Valley',
          'Peoria','Phoenix','Queen Creek','Scottsdale','Surprise','Tempe','Tolleson','Wickenburg','Youngtown']

## As a measure, just check to see how close the word distance is among city/town names themselves.

In [35]:
for i in range(len(cities)):
    print(cities[i],best_match(cities[i],cities[:i]))

('Apache Junction', ('', 0))
('Avondale', ('Apache Junction', 0.2608695652173913))
('Buckeye', ('Apache Junction', 0.18181818181818182))
('Carefree', ('Buckeye', 0.4))
('Cave Creek', ('Carefree', 0.6666666666666666))
('Chandler', ('Avondale', 0.625))
('El Mirage', ('Cave Creek', 0.42105263157894735))
('Fountain Hills', ('Avondale', 0.36363636363636365))
('Gila Bend', ('Avondale', 0.35294117647058826))
('Gilbert', ('Gila Bend', 0.625))
('Glendale', ('Avondale', 0.625))
('Goodyear', ('Gilbert', 0.4))
('Guadalupe', ('Avondale', 0.5882352941176471))
('Litchfield Park', ('Cave Creek', 0.4))
('Mesa', ('Glendale', 0.3333333333333333))
('Paradise Valley', ('Avondale', 0.43478260869565216))
('Peoria', ('El Mirage', 0.4))
('Phoenix', ('Peoria', 0.46153846153846156))
('Queen Creek', ('Cave Creek', 0.6666666666666666))
('Scottsdale', ('Avondale', 0.5555555555555556))
('Surprise', ('Peoria', 0.42857142857142855))
('Tempe', ('El Mirage', 0.42857142857142855))
('Tolleson', ('Phoenix', 0.4))
('Wickenb

In [11]:
def best_match(text, match_list, case = False):
    '''Matches text to match_list by finding the most similar element of match_list and returning
    the element along with the score from 0 to 1 of how good the match is.  The case keyword allows one
    to specify if the matching should be case sensitive or not, default is case=False so that case
    is ignored'''
    
    from difflib import SequenceMatcher
    max_match = 0
    max_pos = 0
    best_match = ''
    for compare in match_list:
        if case == False:
            match_score = SequenceMatcher(None, text.lower(), compare.lower()).ratio()
        elif case == True:
            match_score = SequenceMatcher(None, text, compare).ratio()
        if match_score > max_match:
            max_match = match_score
            best_match = compare
        else:
            continue
    max_score = max_match
    return (best_match, max_score)

## Testing and implementing the best_match function on the csv data

for city_name in city_data_vals:
    match,score = best_match(city_name, cities)
    if score > 0.6:
        print("Match {} to {}".format(city_name, match))
    else:
        print("No match found for {}".format(city_name))


## Now to run the csv data and send out to files

In [12]:
unmatched = []
for i in range(len(ways_tags)):
    if ways_tags['key'][i] == 'city':
        match, score = best_match(ways_tags['value'][i], cities, case=False)
        if score > 0.6:
            ways_tags.loc[i,'value'] = match
        else:
            unmatched.append(ways_tags['value'][i])

In [7]:
set(unmatched)

{'Fort McDowell',
 'Gold Canyon',
 'Higley',
 'Laveen',
 'Laveen Village',
 'Luke AFB',
 'Luke AFB, Waddell',
 'Mayer',
 'Morristown',
 'Rio Verde',
 'San Tan Valley',
 'Sun City',
 'Sun City West',
 'Sun Lakes',
 'Superstition Mountain',
 'sun City West'}

In [13]:
unmatched = []
for i in range(len(nodes_tags)):
    if nodes_tags['key'][i] == 'city':
        match, score = best_match(nodes_tags['value'][i], cities, case=False)
        if score > 0.6:
            nodes_tags.loc[i,'value'] = match
        else:
            unmatched.append(nodes_tags['value'][i])

In [9]:
ways_tags[ways_tags['value'] == "San Diego"]

,Unnamed: 0,Unnamed: 0.1,id,key,value,type,county
198777,198777,198777,5618435,name_base,San Diego,tiger,NaN


# Now manipulate the 'Street' vs 'St' type inconsistencies

In [14]:
import string
CONVENTIONS = [['West','W'],['North','N'],["South","S"],['East','E'],['Street',"St"],["Road","Rd"],["Avenue","Av","Ave"],
                            ["Place","Pl"],["Boulevard","Blvd"],["Trail","Tr"],["Place","Pl"],["Highway","Hwy","Hw","Hy"],
                            ["Parkway","Pkwy","Pw"]]
num_conventions = len(CONVENTIONS)

def conv_street(street_name):
    '''Convert abbreviations in the street names to their full word counter part as specified by CONVENTIONS'''
    split_up = street_name.split(" ")
    for i in range(len(split_up)):
        for j in range(num_conventions):
            #Strip punctuation and compare to naming conventions list
            if split_up[i].translate(None, string.punctuation) in CONVENTIONS[j]:
                split_up[i] = CONVENTIONS[j][0]
    #piece the street string back together with the change
    whole = ' '.join(c for c in split_up)
    return whole

def conv_df(df):
    #This function takes a pandas dataframe which contains a 'key' column with 'street' values and 
    # converts the values according to the conv_street function
    for i in range(len(df)):
        if df.loc[i,'key'] == 'street':
            df.loc[i,'value'] = conv_street(df.loc[i,'value'])
    return df
        

In [15]:
nodes_tags = conv_df(nodes_tags)
ways_tags = conv_df(ways_tags)

## Create uniformity in the 'key' = 'state' values that refer to an address

In [16]:
AZ_reps = ['A', 'AS', 'AZ', 'AZ (Arizona)', 'AZZ', 'Arizona', 'Az', 'az']
for i in range(len(ways_tags)):
    if ways_tags.loc[i,'key'] == 'state' and ways_tags.loc[i,'type'] == 'addr' and ways_tags.loc[i,'value'] in AZ_reps:
        ways_tags.loc[i,'value'] = 'AZ'

        
for i in range(len(nodes_tags)):
    if nodes_tags.loc[i,'key'] == 'state' and nodes_tags.loc[i,'type'] == 'addr' and nodes_tags.loc[i,'value'] in AZ_reps:
        nodes_tags.loc[i,'value'] = 'AZ'

## Check for what inconsistencies may exist in the 'key'='county' data in *_tags data

In [9]:
set(nodes_tags[nodes_tags['key'] == 'county']['value'])

{'Maricopa'}

In [10]:
set(ways_tags[ways_tags['key'] == 'county']['value'])

{'Gila, AZ',
 'Maricopa',
 'Maricopa, AZ',
 'Maricopa, AZ:Yavapai, AZ',
 'Pinal, AZ'}

#### This will be easy enought to convert to just contain the county name and not the state abbreviation which should  be in another field for a database.  It will just take some time.

In [17]:
for i in range(len(ways_tags)):
    if ways_tags.loc[i,'key'] == 'county':
        split = ways_tags.loc[i,'value'].split(",")
        if 'Maricopa' == split[0]:
            ways_tags.loc[i,'county'] = 'Maricopa'
        elif 'Gila' == split[0]:
            ways_tags.loc[i,'county'] = 'Gila'
        elif 'Pinal' == split[0]:
            ways_tags.loc[i,'county'] = 'Pinal'       

## New since first submission

In [12]:

def unif_postcodes(code):
    #Function parses code from variations #####, #####-####, AZ ##### and will be converted to 5 digit only #####
    code = code.split('-')[0]
    if len(code.split(' ')) == 2:
        return code.split(' ')[1]
    else:
        return code
    
##TESTING
#print('44444',unif_postcodes('44444'))
#print('AZ 44444',unif_postcodes('AZ 44444'))
#print('44444-4444',unif_postcodes('44444-4444'))
#print('AZ 44444-4444',unif_postcodes('AZ 44444-4444'))

def process_df(df):
    #Apply unif_postcode to a dataframe with particular columns
    for i in range(len(df)):
        if df['key'][i] == 'postcode':
            df.loc[i,'value'] = unif_postcodes(df.loc[i,'value'])
    return df

In [13]:
ways_tags = process_df(ways_tags)
nodes_tags = process_df(nodes_tags)

In [15]:
nodes_tags.to_csv("nodes_tags.csv")
ways_tags.to_csv("ways_tags.csv")

In [3]:
## Create the OSM File to turn in with project
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "Phx_metro.osm"  # Replace this with your osm file
SAMPLE_FILE = "Phx_metro_smaller.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')